# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [6]:
%help


# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %timeout            Int           The number of minutes after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session.
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0, 3.0 and 4.0. 
                                      Default: 2.0.
    %reconnect          String        Specify a live session ID to switch/reconnect to the sessions.
----

## Selecting Session Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %session_type       String        Specify a session_type to be used. Supported values: streaming and etl.
----

## Glue Config Magic 
*(common across all session types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
    %%tags        Dictionary          Specify a json-formatted dictionary consisting of tags to use in the session.
    
    %%assume_role Dictionary, String  Specify a json-formatted dictionary or an IAM role ARN string to create a session 
                                      for cross account access.
                                      E.g. {valid arn}
                                      %%assume_role 
                                      'arn:aws:iam::XXXXXXXXXXXX:role/AWSGlueServiceRole' 
                                      E.g. {credentials}
                                      %%assume_role
                                      {
                                            "aws_access_key_id" : "XXXXXXXXXXXX",
                                            "aws_secret_access_key" : "XXXXXXXXXXXX",
                                            "aws_session_token" : "XXXXXXXXXXXX"
                                       }
----

                                      
## Magic for Spark Sessions (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
    %matplot      Matplotlib figure   Visualize your data using the matplotlib library.
                                      E.g. 
                                      import matplotlib.pyplot as plt
                                      # Set X-axis and Y-axis values
                                      x = [5, 2, 8, 4, 9]
                                      y = [10, 4, 8, 5, 2]
                                      # Create a bar chart 
                                      plt.bar(x, y) 
                                      # Show the plot
                                      %matplot plt    
    %plotly            Plotly figure  Visualize your data using the plotly library.
                                      E.g.
                                      import plotly.express as px
                                      #Create a graphical figure
                                      fig = px.line(x=["a","b","c"], y=[1,3,2], title="sample figure")
                                      #Show the figure
                                      %plotly fig

  
                
----



####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: d34a7576-e055-4c0d-88bf-18d4246e5c53
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session d34a7576-e055-4c0d-88bf-18d4246e5c53 to get into ready status...
Session d34a7576-e055-4c0d-88bf-18d4246e5c53 ha

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [5]:
#dyf = glueContext.create_dynamic_frame.from_catalog(database='database_name', table_name='table_name')
#dyf.printSchema()

In [2]:
dyf_emp = glueContext.create_dynamic_frame.from_options(
    connection_type = "s3",
    connection_options = {
        "paths": ["s3://practice-spark-shu/employees_120.csv"]
    },
    format = "CSV",
    format_options = {
    "withHeader" : True
    }

)

In [3]:
dyf_emp = dyf_emp.resolveChoice(specs=[("emp_id","cast:int"),
                                      ("salary","cast:double"),])

In [4]:
dyf_emp.printSchema()

root
|-- emp_id: int
|-- name: string
|-- department: string
|-- salary: double
|-- join_date: string
|-- city: string


In [5]:
df_emp = dyf_emp.toDF()

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [6]:
df_emp.show()

+------+------+----------+--------+----------+---------+
|emp_id|  name|department|  salary| join_date|     city|
+------+------+----------+--------+----------+---------+
|    12| Emp12|        IT|115195.0|2022-04-21|    Delhi|
|    32| Emp32|       Ops|113663.0|2018-12-08|     Pune|
|    52| Emp52|   Finance|131504.0|2023-05-25|Bangalore|
|    72| Emp72|     Sales| 54813.0|2018-07-22|Bangalore|
|    92| Emp92|     Sales| 95947.0|2020-06-15|   Mumbai|
|   112|Emp112|        HR|132802.0|2019-01-23|     Pune|
|    13| Emp13|       Ops| 43973.0|2022-01-11|Bangalore|
|    33| Emp33|        IT| 62829.0|2020-12-04|     Pune|
|    53| Emp53|       Ops| 99648.0|2019-11-10|Bangalore|
|    73| Emp73|       Ops|143634.0|2020-05-21|    Delhi|
|    93| Emp93|   Finance|128704.0|2022-10-13|    Delhi|
|   113|Emp113|        IT| 30105.0|2018-10-24|     Pune|
|    14| Emp14|        HR| 50365.0|2019-12-11|Bangalore|
|    34| Emp34|     Sales|115460.0|2023-01-30|    Delhi|
|    54| Emp54|        HR| 7368

In [7]:
dyf_sales = glueContext.create_dynamic_frame.from_options(
    connection_type = 's3',
    connection_options = {
        "paths":["s3://practice-spark-shu/sales_120.csv"]
        
    },
    format = "CSV",
    format_options = {
        "withHeader" : True
    }
)

In [8]:
dyf_sales = dyf_sales.resolveChoice(specs=[("order_id","cast:int"),
                                          ("price","cast:double"),
                                          ("quantity","cast:int")])

In [9]:
dyf_sales.printSchema()

root
|-- order_id: int
|-- customer_id: string
|-- product: string
|-- price: double
|-- quantity: int
|-- order_date: string


In [10]:
df_sales = dyf_sales.toDF()

In [11]:
df_sales.show()

+--------+-----------+----------+------+--------+----------+
|order_id|customer_id|   product| price|quantity|order_date|
+--------+-----------+----------+------+--------+----------+
|      12|         C7|     Phone| 800.0|       2|2024-01-03|
|      32|         C4|    Tablet| 800.0|       1|2024-01-28|
|      52|         C8|Headphones|1200.0|       5|2024-01-05|
|      72|         C4|    Tablet| 100.0|       3|2024-01-25|
|      92|         C8|    Laptop|1200.0|       4|2024-01-06|
|     112|         C3|     Watch| 300.0|       3|2024-01-22|
|      13|         C1|     Watch| 500.0|       3|2024-01-12|
|      33|         C6|Headphones| 500.0|       2|2024-01-15|
|      53|        C20|Headphones| 200.0|       3|2024-01-12|
|      73|        C12|    Tablet| 800.0|       3|2024-01-04|
|      93|         C4|Headphones| 200.0|       4|2024-01-26|
|     113|        C13|Headphones| 300.0|       2|2024-01-31|
|      14|        C11|     Watch| 500.0|       5|2024-01-03|
|      34|        C17|  

In [12]:
dyf_tran = glueContext.create_dynamic_frame.from_options(
    connection_type = "s3",
    connection_options = {
        "paths":["s3://practice-spark-shu/transactions_120.csv"]
    },
    format = "CSV",
    format_options = {"withHeader":True}
)

In [13]:
dyf_tran = dyf_tran.resolveChoice(specs = [("amount","cast:double")])

In [14]:
dyf_tran.printSchema()

root
|-- txn_id: string
|-- user_id: string
|-- amount: double
|-- status: string
|-- txn_date: string


In [15]:
df_tran = dyf_tran.toDF()

In [16]:
df_tran.show()

+------+-------+------+-------+----------+
|txn_id|user_id|amount| status|  txn_date|
+------+-------+------+-------+----------+
|   T12|    U15|1162.0|SUCCESS|2024-02-14|
|   T32|    U13|1256.0| FAILED|2024-02-12|
|   T52|    U11| 416.0| FAILED|2024-02-21|
|   T72|    U13| 954.0|SUCCESS|2024-02-11|
|   T92|     U2|1781.0| FAILED|2024-02-15|
|  T112|    U19|1882.0| FAILED|2024-02-11|
|   T13|    U11| 734.0|SUCCESS|2024-02-12|
|   T33|    U11| 890.0| FAILED|2024-02-17|
|   T53|     U6|1246.0| FAILED|2024-02-09|
|   T73|    U19| 783.0|SUCCESS|2024-02-08|
|   T93|     U2|1692.0| FAILED|2024-02-12|
|  T113|     U6| 604.0| FAILED|2024-02-01|
|   T14|     U4|1314.0| FAILED|2024-02-19|
|   T34|    U18|1498.0| FAILED|2024-02-04|
|   T54|     U4|1314.0|SUCCESS|2024-02-04|
|   T74|    U17| 602.0| FAILED|2024-02-13|
|   T94|    U19| 513.0| FAILED|2024-02-20|
|  T114|     U4|1000.0| FAILED|2024-02-17|
|   T15|    U16| 541.0| FAILED|2024-02-10|
|   T35|    U20|1098.0|SUCCESS|2024-02-07|
+------+---

In [49]:
from pyspark.sql.functions import col, lit, when, current_date, to_date, to_timestamp, sum, trim, avg,round, row_number,dense_rank,rank, month,year,day,countDistinct,lag,lead
from pyspark.sql.window import Window

In [18]:
dfsales1 = df_sales.withColumn("revenue",(col("quantity")*col("price")))

In [19]:
dfsales1.show()

+--------+-----------+----------+------+--------+----------+-------+
|order_id|customer_id|   product| price|quantity|order_date|revenue|
+--------+-----------+----------+------+--------+----------+-------+
|      12|         C7|     Phone| 800.0|       2|2024-01-03| 1600.0|
|      32|         C4|    Tablet| 800.0|       1|2024-01-28|  800.0|
|      52|         C8|Headphones|1200.0|       5|2024-01-05| 6000.0|
|      72|         C4|    Tablet| 100.0|       3|2024-01-25|  300.0|
|      92|         C8|    Laptop|1200.0|       4|2024-01-06| 4800.0|
|     112|         C3|     Watch| 300.0|       3|2024-01-22|  900.0|
|      13|         C1|     Watch| 500.0|       3|2024-01-12| 1500.0|
|      33|         C6|Headphones| 500.0|       2|2024-01-15| 1000.0|
|      53|        C20|Headphones| 200.0|       3|2024-01-12|  600.0|
|      73|        C12|    Tablet| 800.0|       3|2024-01-04| 2400.0|
|      93|         C4|Headphones| 200.0|       4|2024-01-26|  800.0|
|     113|        C13|Headphones| 

In [20]:
dfsales1.groupBy("customer_id").agg(sum(col("revenue")).alias("revenue by cust")).show()

+-----------+---------------+
|customer_id|revenue by cust|
+-----------+---------------+
|         C2|        14600.0|
|         C1|        14500.0|
|         C7|        19800.0|
|        C14|        15900.0|
|        C10|         9600.0|
|         C9|         4500.0|
|         C6|         8200.0|
|        C20|        13300.0|
|        C15|         3200.0|
|         C8|        16600.0|
|        C11|        11100.0|
|         C5|         4200.0|
|        C12|         7900.0|
|        C17|        10300.0|
|        C18|         8000.0|
|         C3|         3600.0|
|         C4|        13100.0|
|        C13|         4900.0|
|        C16|         1500.0|
+-----------+---------------+


In [21]:
dfsales1.groupBy(trim(col("product"))).agg(sum(col("revenue")).alias("revenue by product")).show()

+-------------+------------------+
|trim(product)|revenue by product|
+-------------+------------------+
|       Tablet|           29400.0|
|        Watch|           29000.0|
|       Laptop|           54700.0|
|        Phone|           48500.0|
|   Headphones|           23200.0|
+-------------+------------------+


In [22]:
spark.conf.get("spark.sql.shuffle.partitions")

'16'


In [23]:
dfsales1 = dfsales1.withColumn("order_date", to_date(col("order_date"),"yyyy-MM-dd"))

In [24]:
dfsales1.groupBy("order_date").agg(sum(col("revenue")).alias("sum_by_date")).show()

+----------+-----------+
|order_date|sum_by_date|
+----------+-----------+
|2024-01-06|    10400.0|
|2024-01-07|     3800.0|
|2024-01-08|     7300.0|
|2024-01-09|     6400.0|
|2024-01-10|     3800.0|
|2024-01-11|     7600.0|
|2024-01-12|     5100.0|
|2024-01-13|     7300.0|
|2024-01-14|      300.0|
|2024-01-15|     4700.0|
|2024-01-16|     2000.0|
|2024-01-17|    20900.0|
|2024-01-18|     2000.0|
|2024-01-19|     4500.0|
|2024-01-20|     5400.0|
|2024-01-21|     5500.0|
|2024-01-22|     9200.0|
|2024-01-23|     3300.0|
|2024-01-24|     1000.0|
|2024-01-25|     5700.0|
+----------+-----------+
only showing top 20 rows


In [25]:
dfsales1.groupBy("customer_id").agg(round(avg(col("revenue")),2).alias("avg_revenue_per_cust")).show()

+-----------+--------------------+
|customer_id|avg_revenue_per_cust|
+-----------+--------------------+
|         C2|             2433.33|
|         C1|             1318.18|
|         C7|              2475.0|
|        C14|              1987.5|
|        C10|              2400.0|
|         C9|              1125.0|
|         C6|             1366.67|
|        C20|              1330.0|
|        C15|             1066.67|
|         C8|             1844.44|
|        C11|              2220.0|
|         C5|               700.0|
|        C12|             1316.67|
|        C17|              1287.5|
|        C18|             1142.86|
|         C4|             1871.43|
|         C3|               720.0|
|        C13|              816.67|
|        C16|              1500.0|
+-----------+--------------------+


In [26]:
dfsales1.groupBy("customer_id").agg(sum(col("revenue")).alias("revenue by cust")).filter(col("revenue by cust")>5000).show()

+-----------+---------------+
|customer_id|revenue by cust|
+-----------+---------------+
|         C7|        19800.0|
|         C4|        13100.0|
|         C8|        16600.0|
|         C6|         8200.0|
|        C20|        13300.0|
|        C12|         7900.0|
|         C1|        14500.0|
|        C17|        10300.0|
|        C11|        11100.0|
|         C2|        14600.0|
|        C18|         8000.0|
|        C14|        15900.0|
|        C10|         9600.0|
+-----------+---------------+


In [27]:
w = Window.orderBy(col("revenue_by_cust").desc())
dfsales1.groupBy("customer_id").agg(sum(col("revenue")).alias("revenue_by_cust")).withColumn("rn",row_number().over(w)).filter(col("rn") <= 5).drop("rn").show()

+-----------+---------------+
|customer_id|revenue_by_cust|
+-----------+---------------+
|         C7|        19800.0|
|         C8|        16600.0|
|        C14|        15900.0|
|         C2|        14600.0|
|         C1|        14500.0|
+-----------+---------------+


In [28]:
w = Window.orderBy(col("revenue_by_prod").desc())
dfsales1.groupBy(trim(col("product")).alias("product")).agg(sum(col("revenue")).alias("revenue_by_prod")).withColumn("rn",row_number().over(w)).filter(col("rn") <= 5).drop("rn").show()

+----------+---------------+
|   product|revenue_by_prod|
+----------+---------------+
|    Laptop|        54700.0|
|     Phone|        48500.0|
|    Tablet|        29400.0|
|     Watch|        29000.0|
|Headphones|        23200.0|
+----------+---------------+


In [29]:
w = Window.orderBy(col("revenue_by_prod").desc())
dfsales1.groupBy(trim(col("product")).alias("product")).agg(sum(col("revenue")).alias("revenue_by_prod")).withColumn("rn",dense_rank().over(w)).show()

+----------+---------------+---+
|   product|revenue_by_prod| rn|
+----------+---------------+---+
|    Laptop|        54700.0|  1|
|     Phone|        48500.0|  2|
|    Tablet|        29400.0|  3|
|     Watch|        29000.0|  4|
|Headphones|        23200.0|  5|
+----------+---------------+---+


In [30]:
w = Window.orderBy(col("revenue_by_cust").desc())
dfsales1.groupBy("customer_id").agg(sum(col("revenue")).alias("revenue_by_cust")).withColumn("rn",rank().over(w)).show()

+-----------+---------------+---+
|customer_id|revenue_by_cust| rn|
+-----------+---------------+---+
|         C7|        19800.0|  1|
|         C8|        16600.0|  2|
|        C14|        15900.0|  3|
|         C2|        14600.0|  4|
|         C1|        14500.0|  5|
|        C20|        13300.0|  6|
|         C4|        13100.0|  7|
|        C11|        11100.0|  8|
|        C17|        10300.0|  9|
|        C10|         9600.0| 10|
|         C6|         8200.0| 11|
|        C18|         8000.0| 12|
|        C12|         7900.0| 13|
|        C13|         4900.0| 14|
|         C9|         4500.0| 15|
|         C5|         4200.0| 16|
|         C3|         3600.0| 17|
|        C15|         3200.0| 18|
|        C16|         1500.0| 19|
+-----------+---------------+---+


In [31]:
w = Window.partitionBy("order_date").orderBy("order_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

dfsales1.withColumn("running_revenue", sum(col("revenue")).over(w)).show()

+--------+-----------+----------+------+--------+----------+-------+---------------+
|order_id|customer_id|   product| price|quantity|order_date|revenue|running_revenue|
+--------+-----------+----------+------+--------+----------+-------+---------------+
|     114|         C9|     Phone| 100.0|       5|2024-01-01|  500.0|          500.0|
|      48|         C8|    Tablet| 100.0|       4|2024-01-01|  400.0|          900.0|
|      25|         C5|     Phone| 500.0|       2|2024-01-02| 1000.0|         1000.0|
|      80|        C11|     Phone| 300.0|       2|2024-01-03|  600.0|          600.0|
|      23|        C13|    Tablet| 200.0|       2|2024-01-03|  400.0|         1000.0|
|      12|         C7|     Phone| 800.0|       2|2024-01-03| 1600.0|         2600.0|
|      42|        C14|    Laptop| 200.0|       3|2024-01-03|  600.0|         3200.0|
|     105|         C1|     Watch| 200.0|       2|2024-01-03|  400.0|         3600.0|
|      14|        C11|     Watch| 500.0|       5|2024-01-03| 2500

In [32]:
w = Window.orderBy("order_date").rowsBetween(-2, 0)

dfsales1.withColumn("moving_avg_revenue_3d", avg(col("revenue")).over(w)).show()

+--------+-----------+----------+------+--------+----------+-------+---------------------+
|order_id|customer_id|   product| price|quantity|order_date|revenue|moving_avg_revenue_3d|
+--------+-----------+----------+------+--------+----------+-------+---------------------+
|     114|         C9|     Phone| 100.0|       5|2024-01-01|  500.0|                500.0|
|      48|         C8|    Tablet| 100.0|       4|2024-01-01|  400.0|                450.0|
|      25|         C5|     Phone| 500.0|       2|2024-01-02| 1000.0|    633.3333333333334|
|      12|         C7|     Phone| 800.0|       2|2024-01-03| 1600.0|               1000.0|
|      14|        C11|     Watch| 500.0|       5|2024-01-03| 2500.0|               1700.0|
|      34|        C17|    Laptop| 500.0|       5|2024-01-03| 2500.0|               2200.0|
|      54|        C13|     Watch|1200.0|       1|2024-01-03| 1200.0|   2066.6666666666665|
|      88|        C20|     Phone| 800.0|       5|2024-01-03| 4000.0|   2566.6666666666665|

In [36]:
w = Window.partitionBy("order_id").orderBy(col("order_date").desc())
dfsales1.withColumn("rn",row_number().over(w)).filter(col("rn")>1).show()

+--------+-----------+-------+-----+--------+----------+-------+---+
|order_id|customer_id|product|price|quantity|order_date|revenue| rn|
+--------+-----------+-------+-----+--------+----------+-------+---+
+--------+-----------+-------+-----+--------+----------+-------+---+


In [43]:
dfsales1 = dfsales1.withColumn("month",month(col("order_date")).cast("int"))

In [48]:
dfsales1.select(countDistinct("month")).show()

+---------------------+
|count(DISTINCT month)|
+---------------------+
|                    1|
+---------------------+


In [57]:
dfsales1.withColumn("lagging", lag(col("revenue")).over(Window.partitionBy().orderBy("revenue"))).show()

+--------+-----------+----------+-----+--------+----------+-------+-----+-------+
|order_id|customer_id|   product|price|quantity|order_date|revenue|month|lagging|
+--------+-----------+----------+-----+--------+----------+-------+-----+-------+
|      74|         C7|    Laptop|100.0|       1|2024-01-11|  100.0|    1|   NULL|
|      94|        C12|    Tablet|100.0|       1|2024-01-12|  100.0|    1|  100.0|
|      36|        C18|     Phone|100.0|       1|2024-01-29|  100.0|    1|  100.0|
|      45|        C10|     Phone|100.0|       1|2024-01-13|  100.0|    1|  100.0|
|      85|        C13|     Watch|100.0|       1|2024-01-05|  100.0|    1|  100.0|
|       9|         C8|Headphones|100.0|       2|2024-01-24|  200.0|    1|  100.0|
|      84|        C17|     Phone|100.0|       2|2024-01-16|  200.0|    1|  200.0|
|      72|         C4|    Tablet|100.0|       3|2024-01-25|  300.0|    1|  200.0|
|     117|         C3|Headphones|100.0|       3|2024-01-22|  300.0|    1|  300.0|
|     120|      

In [59]:
dfsales1.withColumn("lagging", lead(col("order_date")).over(Window.partitionBy().orderBy("order_date"))).show()

+--------+-----------+----------+------+--------+----------+-------+-----+----------+
|order_id|customer_id|   product| price|quantity|order_date|revenue|month|   lagging|
+--------+-----------+----------+------+--------+----------+-------+-----+----------+
|     114|         C9|     Phone| 100.0|       5|2024-01-01|  500.0|    1|2024-01-01|
|      48|         C8|    Tablet| 100.0|       4|2024-01-01|  400.0|    1|2024-01-02|
|      25|         C5|     Phone| 500.0|       2|2024-01-02| 1000.0|    1|2024-01-03|
|      42|        C14|    Laptop| 200.0|       3|2024-01-03|  600.0|    1|2024-01-03|
|     105|         C1|     Watch| 200.0|       2|2024-01-03|  400.0|    1|2024-01-03|
|      80|        C11|     Phone| 300.0|       2|2024-01-03|  600.0|    1|2024-01-03|
|      23|        C13|    Tablet| 200.0|       2|2024-01-03|  400.0|    1|2024-01-03|
|      12|         C7|     Phone| 800.0|       2|2024-01-03| 1600.0|    1|2024-01-03|
|      14|        C11|     Watch| 500.0|       5|2024-

In [60]:
dfsales1.withColumn("day",day(col("order_date"))).show()

+--------+-----------+----------+------+--------+----------+-------+-----+---+
|order_id|customer_id|   product| price|quantity|order_date|revenue|month|day|
+--------+-----------+----------+------+--------+----------+-------+-----+---+
|      12|         C7|     Phone| 800.0|       2|2024-01-03| 1600.0|    1|  3|
|      32|         C4|    Tablet| 800.0|       1|2024-01-28|  800.0|    1| 28|
|      52|         C8|Headphones|1200.0|       5|2024-01-05| 6000.0|    1|  5|
|      72|         C4|    Tablet| 100.0|       3|2024-01-25|  300.0|    1| 25|
|      92|         C8|    Laptop|1200.0|       4|2024-01-06| 4800.0|    1|  6|
|     112|         C3|     Watch| 300.0|       3|2024-01-22|  900.0|    1| 22|
|      13|         C1|     Watch| 500.0|       3|2024-01-12| 1500.0|    1| 12|
|      33|         C6|Headphones| 500.0|       2|2024-01-15| 1000.0|    1| 15|
|      53|        C20|Headphones| 200.0|       3|2024-01-12|  600.0|    1| 12|
|      73|        C12|    Tablet| 800.0|       3|202